# Importing Libraries and Data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt

In [35]:
mosquito = pd.read_csv('./west_nile/input/Chicago_WNV.csv')
weather_0607 = pd.read_csv('./west_nile/input/1473571.csv')
weather_0809 = pd.read_csv('./west_nile/input/1473573.csv')
weather_1011 = pd.read_csv('./west_nile/input/1473576.csv')
weather_1213 = pd.read_csv('./west_nile/input/1473577.csv')
weather_1415 = pd.read_csv('./west_nile/input/1473578.csv')
weather_1617 = pd.read_csv('./west_nile/input/1473579.csv')
weather_18 = pd.read_csv('./west_nile/input/1473580.csv')

#  Mosquito Data
### Taking a look at the mosquito data

In [3]:
mosquito.shape

(27054, 13)

In [4]:
mosquito.describe()

,SEASON YEAR,WEEK,TEST ID,NUMBER OF MOSQUITOES,LATITUDE,LONGITUDE
count,27054.000000,27054.000000,27054.000000,27054.000000,22970.000000,22970.000000
mean,2011.922414,31.056738,33926.895284,12.510645,41.835401,-87.687765
std,3.418862,4.480369,7927.693220,15.471264,0.113346,0.076972
min,2007.000000,20.000000,20000.000000,1.000000,41.644720,-87.854473
25%,2009.000000,28.000000,27103.250000,2.000000,41.731447,-87.745420
50%,2012.000000,31.000000,33907.500000,5.000000,41.825398,-87.689417
75%,2015.000000,35.000000,40806.750000,17.000000,41.945961,-87.632955
max,2018.000000,40.000000,47592.000000,77.000000,42.017801,-87.531060


In [6]:
mosquito.isna().sum()

SEASON YEAR                0
WEEK                       0
TEST ID                    0
BLOCK                      0
TRAP                       0
TRAP_TYPE                  0
TEST DATE                  0
NUMBER OF MOSQUITOES       0
RESULT                     0
SPECIES                    0
LATITUDE                4084
LONGITUDE               4084
LOCATION                4084
dtype: int64

In [7]:
mosquito.head()

,SEASON YEAR,WEEK,TEST ID,BLOCK,TRAP,TRAP_TYPE,TEST DATE,NUMBER OF MOSQUITOES,RESULT,SPECIES,LATITUDE,LONGITUDE,LOCATION
0,2011,29,31550,100XX W OHARE AIRPORT,T916,GRAVID,07/25/2011 12:07:00 AM,3,negative,CULEX PIPIENS,NaN,NaN,NaN
1,2016,25,42613,58XX N PULASKI RD,T027,GRAVID,06/22/2016 12:06:00 AM,6,negative,CULEX RESTUANS,41.986320,-87.728378,"(41.986319851449004, -87.72837845617912)"
2,2007,27,20583,15XX W WEBSTER AVE,T045,GRAVID,07/11/2007 03:07:41 AM,4,negative,CULEX PIPIENS/RESTUANS,41.921705,-87.666963,"(41.92170457422864, -87.66696323469388)"
3,2009,38,28275,5XX S CENTRAL AVE,T031,GRAVID,09/25/2009 12:09:00 AM,4,negative,CULEX PIPIENS,41.872873,-87.764737,"(41.87287286249572, -87.7647365320396)"
4,2011,34,32254,58XX N PULASKI RD,T027,GRAVID,08/26/2011 12:08:00 AM,2,negative,CULEX RESTUANS,41.986320,-87.728378,"(41.986319851449004, -87.72837845617912)"


### Cleaning the mosquito data
#### Beginning with some quick fixes

In [9]:
# Changing the columns to be easier to use
mosquito = mosquito.sort_values(by='TEST ID')
mosquito['RESULT'] = mosquito['RESULT'].map(lambda x: 0 if x == 'negative' else 1)
mosquito['TEST DATE'] = pd.to_datetime(mosquito['TEST DATE'])

#### Fixing the missing locations

In [10]:
# Looking at which location information is missing
mosquito[mosquito['LOCATION'].isna()==True]['BLOCK'].unique()

array(['65XX N OAK PARK AVE', '43XX N ASHLAND', '4XX W 127TH',
       '81XX S ASHLAND', '79XX S CHICAGO', '30XX S HOYNE',
       '98XX S AVENUE G', '115XX S AVENUE L', '100XX W OHARE',
       '100XX W OHARE AIRPORT'], dtype=object)

In [13]:
# Filling in all of the missing lat and long data using Google Maps
for i in mosquito[mosquito['BLOCK'] == '65XX N OAK PARK AVE'].index:
    mosquito.loc[i,['LATITUDE', 'LONGITUDE']] = [42.000048, -87.795637]
for i in mosquito[mosquito['BLOCK'] == '43XX N ASHLAND'].index:
    mosquito.loc[i,['LATITUDE', 'LONGITUDE']] = [41.960938, -87.669239]
for i in mosquito[mosquito['BLOCK'] == '4XX W 127TH'].index:
    mosquito.loc[i,['LATITUDE', 'LONGITUDE']] = [41.663323, -87.633490]
for i in mosquito[mosquito['BLOCK'] == '79XX S CHICAGO'].index:
    mosquito.loc[i,['LATITUDE', 'LONGITUDE']] = [41.752418, -87.587062]
for i in mosquito[mosquito['BLOCK'] == '30XX S HOYNE'].index:
    mosquito.loc[i,['LATITUDE', 'LONGITUDE']] = [41.836610, -87.677720]
for i in mosquito[mosquito['BLOCK'] == '98XX S AVENUE G'].index:
    mosquito.loc[i,['LATITUDE', 'LONGITUDE']] = [41.717151, -87.531634]
for i in mosquito[mosquito['BLOCK'] == '115XX S AVENUE L'].index:
    mosquito.loc[i,['LATITUDE', 'LONGITUDE']] = [41.685640, -87.536430]
for i in mosquito[mosquito['BLOCK'] == '100XX W OHARE'].index:
    mosquito.loc[i,['LATITUDE', 'LONGITUDE']] = [41.977498, -87.881006]
for i in mosquito[mosquito['BLOCK'] == '100XX W OHARE AIRPORT'].index:
    mosquito.loc[i,['LATITUDE', 'LONGITUDE']] = [41.978969, -87.902679]
for i in mosquito[mosquito['BLOCK'] == '81XX S ASHLAND'].index:
    mosquito.loc[i,['LATITUDE', 'LONGITUDE']] = [41.745861, -87.663463]

In [15]:
# Checking that it is done
mosquito.isna().sum()

SEASON YEAR                0
WEEK                       0
TEST ID                    0
BLOCK                      0
TRAP                       0
TRAP_TYPE                  0
TEST DATE                  0
NUMBER OF MOSQUITOES       0
RESULT                     0
SPECIES                    0
LATITUDE                   0
LONGITUDE                  0
LOCATION                4084
dtype: int64

# Weather Data
### Grouping the weather data

In [36]:
# Combining all of the weather data for each day
weather_0607 = weather_0607.groupby('DATE', as_index=False).mean()
weather_0809 = weather_0809.groupby('DATE', as_index=False).mean()
weather_1011 = weather_1011.groupby('DATE', as_index=False).mean()
weather_1213 = weather_1213.groupby('DATE', as_index=False).mean()
weather_1415 = weather_1415.groupby('DATE', as_index=False).mean()
weather_1617 = weather_1617.groupby('DATE', as_index=False).mean()
weather_18 = weather_18.groupby('DATE', as_index=False).mean()

In [37]:
# Combining all of the weather data into one data frame
weather = pd.concat([weather_0607, weather_0809, weather_1011, weather_1213, weather_1415, weather_1617, weather_18])
weather.reset_index(drop=True, inplace=True)

### Looking at the weather data

In [38]:
weather.shape

(4644, 6)

In [39]:
weather.describe()

,AWND,PRCP,TAVG,TMAX,TMIN
count,4642.000000,4644.000000,1915.000000,4644.000000,4644.000000
mean,9.026103,0.118150,51.086684,60.080216,42.106422
std,3.530508,0.277222,20.121984,20.904471,19.048272
min,1.713333,0.000000,-8.000000,-0.333333,-18.400000
25%,6.336667,0.000093,36.000000,42.800000,28.300000
50%,8.573333,0.006850,53.000000,62.000000,42.700000
75%,11.260000,0.094230,69.000000,78.900000,58.800000
max,24.533333,3.761504,87.000000,102.111111,79.666667


In [40]:
weather.isna().sum()

DATE       0
AWND       2
PRCP       0
TAVG    2729
TMAX       0
TMIN       0
dtype: int64

In [41]:
weather.head()

,DATE,AWND,PRCP,TAVG,TMAX,TMIN
0,2006-01-01,8.276667,0.006000,NaN,42.0,29.0
1,2006-01-02,12.226667,0.210667,NaN,43.4,37.2
2,2006-01-03,6.786667,0.200000,NaN,42.3,37.7
3,2006-01-04,11.933333,0.015000,NaN,43.0,36.1
4,2006-01-05,12.226667,0.006250,NaN,39.6,31.9


#### Fixing the NaNs in the data set

In [43]:
# Using the average of max and min temp for days with missing average temp
for i in weather[weather['TAVG'].isna()].index:
    weather.loc[i, 'TAVG'] = weather.loc[i, ['TMAX', 'TMIN']].sum()/2

In [44]:
# Correcting winding speed data using weather.gov
weather.loc[weather[weather['DATE']=='2018-09-18'].index, 'AWND'] = 8.3
weather.loc[weather[weather['DATE']=='2018-09-17'].index, 'AWND'] = 3.3

# Exporting the Data

In [49]:
# These are commented out so that we don't accidentally run them.
weather.to_csv('./assets/weather')
mosquito.to_csv('./assets/mosquito_data')